In [4]:
import pandas as pd
import numpy as np
import sklearn
import matt_acquire
import matt_prepare
import pandas as pd
import numpy as np
import sklearn
import matt_acquire
from matt_acquire import get_zillow_data
from sklearn.preprocessing import MinMaxScaler
from scipy import stats


from sklearn.model_selection import train_test_split

from matt_acquire import get_zillow_data
from matt_prepare import prep_zillow_data


In [5]:
# Pull in our data both scaled and unscaled

In [6]:
def prep_zillow_data(target):
    df = get_zillow_data()
    df = df[['calculatedfinishedsquarefeet', 'bedroomcnt', 'bathroomcnt', 'lotsizesquarefeet', 'taxvaluedollarcnt']]
    df.dropna(inplace=True)

    train_validate, test = train_test_split(df, test_size=.2, random_state=123)

    train, validate = train_test_split(train_validate, test_size=.3, random_state=123)

    X_train = train.drop(columns=[target])
    y_train = train[target]
    
    X_validate = validate.drop(columns=[target])
    y_validate = validate[target]
    
    X_test = test.drop(columns=[target])
    y_test = test[target]
    
    scaler = sklearn.preprocessing.MinMaxScaler()

    scaler.fit(X_train)

    X_train_scaled = pd.DataFrame(scaler.transform(X_train))
    X_validate_scaled = pd.DataFrame(scaler.transform(X_validate))
    X_test_scaled = pd.DataFrame(scaler.transform(X_test))


    X_train_scaled.rename(columns={0: 'Unit_Sq_ft', 1: 'Bedroom_Count', 2: 'Bathroom_Count', 3: 'lotsizesquarefeet' }, inplace=True)
    X_validate_scaled.rename(columns={0: 'Unit_Sq_ft', 1: 'Bedroom_Count', 2: 'Bathroom_Count', 3: 'lotsizesquarefeet'}, inplace=True)
    X_test_scaled.rename(columns={0: 'Unit_Sq_ft', 1: 'Bedroom_Count', 2: 'Bathroom_Count', 3: 'lotsizesquarefeet'}, inplace=True)

    return train, y_train, y_validate, y_test, X_train_scaled, X_train, X_validate, X_test, X_validate_scaled, X_test_scaled

In [7]:
train, y_train, y_validate, y_test, X_train_scaled, X_train, X_validate, X_test, X_validate_scaled, X_test_scaled = prep_zillow_data('taxvaluedollarcnt')

In [8]:
# Hypothesis Test

In [9]:
alpha = .05

In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7266 entries, 6531 to 10999
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   calculatedfinishedsquarefeet  7266 non-null   float64
 1   bedroomcnt                    7266 non-null   float64
 2   bathroomcnt                   7266 non-null   float64
 3   lotsizesquarefeet             7266 non-null   float64
dtypes: float64(4)
memory usage: 283.8 KB


# Pearson's

Ho = There is no linear correlation between calculated finished square feet and home value.

H1 = There is a linear correlation between calculated finished square feet and home value.

In [11]:
x = X_train['calculatedfinishedsquarefeet']
y = y_train
r, p = stats.pearsonr(x, y)
print(f'r = {r:.5f}')
print(f'p = {p:.1f}')
p < alpha

r = 0.59134
p = 0.0


True

- Since p is less than alpha, we reject our null hypothesis that there is no linear correlation between calculated finished square feet and home value.

Ho = There is no linear correlation between bedroom count and home value.

H1 = There is a linear correlation between bedroom count and home value.

In [12]:
x = X_train['bedroomcnt']
y = y_train
r, p = stats.pearsonr(x, y)
print(f'r = {r:.5f}')
print(f'p = {p:.1f}')
p < alpha

r = 0.26055
p = 0.0


True

- Since p is less than alpha, we reject our null hypothesis that there is no linear correlation between bedroom count and home value.

Ho = There is no linear correlation between bathroom count and home value.

H1 = There is a linear correlation between bathroom count and home value.

In [13]:
x = X_train['bathroomcnt']
y = y_train
r, p = stats.pearsonr(x, y)
r, p = stats.pearsonr(x, y)
print(f'r = {r:.5f}')
print(f'p = {p:.1f}')
p < alpha

r = 0.45501
p = 0.0


True

- Since p is less than alpha, we reject our null hypothesis that there is no linear correlation between bathroom count and home value.

Ho = There is no linear correlation between lot size and home value.

H1 = There is a linear correlation between lot size and home value.

In [14]:
x = X_train['lotsizesquarefeet']
y = y_train
r, p = stats.pearsonr(x, y)
print(f'r = {r:.5f}')
print(f'p = {p:.1f}')
p < alpha

r = -0.03342
p = 0.0


True

- Since p is less than alpha, we reject our null hypothesis that there is no linear correlation between bathroom count and home value.

# T-Test

Ho - The value of units with three or more bathrooms is equal to or less than the value of units with less than three bathrooms. 

H1 - The value of units with three or more bathrooms is greater than the value of units with less than three bathrooms.

In [18]:
x1 = train[train.bathroomcnt > 2].taxvaluedollarcnt
x2 = train[train.bathroomcnt <= 2].taxvaluedollarcnt

In [25]:
t, p = stats.ttest_ind(x1, x2)
print(f't = {t:.5f}')
print (f'p = {p / 2}')

t = 10.06640
p = 5.5569806575727444e-24


We reject the null hypothesis that there is no significant difference in home value for units with more than two bathrooms vs units with two or less bathrooms.

Ho - The value of units with five or more bedrooms is equal to or less than the value of units with less than five bedrooms. 

H1 - The value of units with five or more bedrooms is greater than the value of units with less than five bedrooms.

In [28]:
x1 = train[train.bedroomcnt > 4].taxvaluedollarcnt
x2 = train[train.bedroomcnt <= 4].taxvaluedollarcnt

In [29]:
t, p = stats.ttest_ind(x1, x2)
print(f't = {t:.5f}')
print (f'p = {p / 2}')

t = 24.03787
p = 3.195724679557474e-123


- Since p is less than alpha, we reject our null hypothesis that there is no linear correlation between bedroom count and home value.